<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-03 11:23:59
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 49.08 K (0.36%)
Current PnL: -13.09 L (-9.12%)
CY Booked + Current PnL: -6.35 L (-4.43%)
-------------------
Total profit:  4.81 L
Total loss:  -17.90 L
-------------------
Total Booked + Current PnL: 20.63 L (17.7%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.93%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.52 L (52.28%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-18.25,H-LC,87.50,112688.0,8376.0,9162.0,0.85,8.03,8.13,16.81,27.0,0.91,0.82,25.36,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,93.75,133785.0,-8694.0,9311.0,-0.70,-6.10,6.96,0.44,243.0,-0.93,0.98,57.90,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,42.71,102574.0,24212.0,9816.0,0.06,30.90,9.57,43.43,77.0,2.47,0.75,37.86,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-17.29,H-LC,85.42,112695.0,9129.0,11585.0,0.97,8.81,10.28,20.00,30.0,0.79,0.82,24.13,X40N,BTT,HEALTHCARE
88,UNIONBANK,123.87,163.00,-1.39,M-LC,31.25,172528.0,31688.0,12802.0,-1.08,22.50,7.42,31.59,89.0,2.48,1.26,49.87,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-30.08,L-SC,100.00,141009.0,-833.0,57405.0,-0.96,-0.59,40.71,39.88,259.0,-0.01,1.03,42.90,XY24,NTT,POWER
38,HEROMOTOCO,4311.81,6039.03,-6.67,H-MC,45.83,150045.0,-868.0,61323.0,1.02,-0.58,40.87,40.06,102.0,-0.01,1.10,22.73,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,HONAUT,42646.15,58357.33,-15.36,X-MC,59.38,121980.0,-5958.0,53098.0,4.75,-4.66,43.53,36.84,56.0,-0.11,0.89,24.88,X40N,ATH,ELECTRICAL
15,BLUESTARCO,1646.70,2337.55,-2.21,H-SC,97.92,183470.0,18800.0,50289.0,4.48,11.42,27.41,41.95,121.0,0.37,1.34,19.74,X40N,ATH,AC
93,VOLTAS,1278.28,1918.49,-24.10,H-MC,89.58,206700.0,14958.0,81068.0,3.39,7.80,39.22,50.08,100.0,0.18,1.51,15.13,XY25,ATH,AC
36,HAVELLS,1588.02,2069.16,-4.46,X-MC,61.98,246901.0,-830.0,75897.0,2.08,-0.34,30.74,30.30,53.0,-0.01,1.81,12.91,X40,ATH,ELECTRICAL
59,MEDANTA,1064.65,1486.00,7.53,X-SC,15.62,229400.0,23923.0,57396.0,1.93,11.64,25.02,39.58,65.0,0.42,1.68,22.93,XY24,NTT,HEALTHCARE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,DMART,3764.92,5201.00,-16.54,X-LC,64.58,209671.0,25190.0,45184.0,-2.57,13.65,21.55,38.14,15.0,0.56,1.53,25.71,X40N,NTT,FMCG
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16429.0,-33164.0,2086.0,-2.17,-66.87,12.70,-62.66,NaN,-15.90,0.12,10.18,XY24,ATH,MISC
44,ICICIPRULI,600.83,790.00,-9.47,H-MC,37.50,146154.0,10366.0,32388.0,-1.84,7.63,22.16,31.48,119.0,0.32,1.07,20.68,X40,ATH,INSURANCE
11,BANDHANBNK,214.25,400.00,-28.38,H-SC,80.21,214020.0,-34510.0,249975.0,-1.80,-13.89,116.80,86.70,129.0,-0.14,1.57,41.59,XY24,NTT,BANKS
32,ENRIN,1377.95,2975.90,NaN,NaN,70.83,72830.0,38381.0,1566.0,-1.28,111.42,2.15,115.97,NaN,24.51,0.53,11.37,AR,ATH,ELECTRICAL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,93.75,133785.0,-8694.0,9311.0,-0.70,-6.10,6.96,0.44,243.0,-0.93,0.98,57.90,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.70,M-SC,8.33,147768.0,-23373.0,23318.0,1.92,-13.66,15.78,-0.03,189.0,-1.00,1.08,6.20,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-213.70,H-SC,95.83,82056.0,-13260.0,13301.0,0.19,-13.91,16.21,0.04,154.0,-1.00,0.60,61.85,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148998.0,-25561.0,25598.0,0.66,-14.64,17.18,0.02,127.0,-1.00,1.09,29.20,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.14,X-MC,17.71,87052.0,-15600.0,15600.0,0.53,-15.20,17.92,-0.00,58.0,-1.00,0.64,32.33,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,7.29,47873.0,6618.0,4189.0,0.21,16.04,8.75,26.19,1.0,1.58,0.35,12.71,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,43.75,266947.0,-28987.0,122902.0,-0.03,-9.80,46.04,31.73,1.0,-0.24,1.95,6.85,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,63.54,291000.0,29399.0,116225.0,0.94,11.24,39.94,55.67,2.0,0.25,2.13,18.03,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.03,X-LC,28.12,259840.0,-10507.0,45134.0,0.56,-3.89,17.37,12.81,6.0,-0.23,1.90,8.67,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.33,X-LC,35.42,281338.0,15912.0,40991.0,0.70,5.99,14.57,21.44,9.0,0.39,2.06,12.56,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,31.25,172528.0,31688.0,12802.0,-1.08,22.50,7.42,31.59,89.0,2.48,1.26,49.87,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.03,M-LC,42.71,102574.0,24212.0,9816.0,0.06,30.90,9.57,43.43,77.0,2.47,0.75,37.86,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,70.83,72830.0,38381.0,1566.0,-1.28,111.42,2.15,115.97,NaN,24.51,0.53,11.37,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,96.87,M-MC,65.62,236517.0,11555.0,156101.0,-0.09,5.14,66.00,74.53,185.0,0.07,1.73,37.32,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,27.10,H-SC,72.92,130070.0,4773.0,54434.0,0.26,3.81,41.85,47.25,139.0,0.09,0.95,62.73,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.32,H-SC,44.79,119375.0,17371.0,27647.0,0.77,17.03,23.16,44.14,124.0,0.63,0.87,36.04,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,42.71,102574.0,24212.0,9816.0,0.06,30.90,9.57,43.43,77.0,2.47,0.75,37.86,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,31.25,172528.0,31688.0,12802.0,-1.08,22.50,7.42,31.59,89.0,2.48,1.26,49.87,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,70.83,72830.0,38381.0,1566.0,-1.28,111.42,2.15,115.97,NaN,24.51,0.53,11.37,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,90.62,205374.0,33753.0,128318.0,0.73,19.67,62.48,94.43,198.0,0.26,1.50,51.58,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,82.29,167245.0,16300.0,93155.0,1.03,10.80,55.70,72.51,69.0,0.17,1.22,15.20,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.70,H-SC,84.38,136369.0,13033.0,121791.0,0.14,10.57,89.31,109.32,122.0,0.11,1.00,34.72,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,27.10,H-SC,72.92,130070.0,4773.0,54434.0,0.26,3.81,41.85,47.25,139.0,0.09,0.95,62.73,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68256.0,-45293.0,85347.0,0.08,-39.89,125.04,35.27,261.0,-0.53,0.50,67.06,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.70,H-SC,66.67,92550.0,-5430.0,28163.0,0.28,-5.54,30.43,23.21,137.0,-0.19,0.68,34.23,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,98.96,100495.0,-468.0,94998.0,-0.41,-0.46,94.53,93.63,64.0,-0.00,0.73,49.69,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.03,M-LC,42.71,102574.0,24212.0,9816.0,0.06,30.90,9.57,43.43,77.0,2.47,0.75,37.86,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-355.77,H-SC,4.69,108602.0,-24684.0,109275.0,0.32,-18.52,100.62,63.47,148.0,-0.23,0.79,82.04,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-30.08,L-SC,100.00,141009.0,-833.0,57405.0,-0.96,-0.59,40.71,39.88,259.0,-0.01,1.03,42.90,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,98.96,100495.0,-468.0,94998.0,-0.41,-0.46,94.53,93.63,64.0,-0.00,0.73,49.69,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-2.21,H-SC,97.92,183470.0,18800.0,50289.0,4.48,11.42,27.41,41.95,121.0,0.37,1.34,19.74,X40N,ATH,AC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68256.0,-45293.0,85347.0,0.08,-39.89,125.04,35.27,261.0,-0.53,0.50,67.06,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-213.70,H-SC,95.83,82056.0,-13260.0,13301.0,0.19,-13.91,16.21,0.04,154.0,-1.00,0.60,61.85,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,42.01
2,50,70.47


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.40
LC    33.97
MC    21.61
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.48
X40      12.26
XY25     12.05
X40N     11.85
XR       10.10
AR        8.23
OX40N     8.21
MH        1.70
X5K       1.48
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.63
M-SC    15.94
H-LC    14.13
X-LC    12.15
H-MC     9.55
X-MC     7.60
M-LC     6.64
X-SC     4.40
M-MC     4.10
L-SC     2.43
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.07,-1.51,35.44
IT,12.14,-9.55,63.38
BANKS,7.88,-5.87,49.07
FINANCE,7.86,-12.17,52.77
MISC,6.62,-14.92,68.24
PAINTS,5.74,-11.28,33.22
ELECTRICAL,5.52,2.36,43.25
HEALTHCARE,4.54,-3.74,34.04
AUTO,4.34,-14.02,64.16


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2695593.0,27
XR,1077470.0,13
AR,1022016.0,10
X40,642840.0,9
X40N,541519.0,11
XY25,470910.0,8
OX40N,308517.0,11
SR,176357.0,2
MH,83258.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1964431.0,20
M-SC,1762717.0,21
H-LC,607496.0,12
H-MC,529991.0,8
X-LC,514011.0,7
X-MC,412681.0,6
M-MC,408799.0,3
X-SC,316312.0,5
L-SC,264459.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      867674.0      6
M-SC       XY24      817284.0      7
H-SC       AR        522315.0      3
M-SC       XR        333391.0      4
M-MC       XY24      298397.0      2
H-MC       XY24      296240.0      3
X-LC       X40       239127.0      2
H-SC       XR        216543.0      3
           X40N      215456.0      4
H-LC       X40N      197633.0      4
M-SC       AR        181364.0      2
           OX40N     164112.0      5
M-LC       XY24      157131.0      3
H-LC       X40       153535.0      3
L-SC       XR        147388.0      2
X-MC       X40       146821.0      2
X-SC       XR        123122.0      1
X-LC       AR        116154.0      1
M-SC       XY25      114238.0      1
X-LC       XY25      113546.0      3
M-MC       XR        110402.0      1
X-MC       XY25      102201.0      1
X-SC       SR         94998.0      1
X-MC       XY24       94961.0      1
M-LC       XR         93155.0      1
H-SC       OX40N      88009.0      3
M-SC       SR         81359.0      1
H-MC       XY25       81068.0      1
H-LC       AR         79628.0      1
M-SC       X40        70969.0      1
H-LC       X200       68405.0      1
           X5K        65465.0      2
H-MC       AR         61323.0      1
L-SC       AR         59666.0      1
           XY24       57405.0      1
X-SC       XY24       57396.0      1
H-SC       MH         54434.0      1
L-MC       XR         53469.0      1
X-MC       X40N       53098.0      1
L-LC       XY25       46384.0      1
X-LC       X40N       45184.0      1
H-LC       XY24       42830.0      1
X-SC       OX40N      40796.0      2
H-MC       X40        32388.0      1
           X40N       30148.0      1
           MH         28824.0      1
X-MC       OX40N      15600.0      1
M-LC       XY25       13473.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 65.0 seconds
